In [6]:
import os
import pandas as pd
import mysql.connector
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [8]:
import chart_studio

chart_studio.tools.set_credentials_file(username = os.environ.get('cs_user'), api_key = os.environ.get('cs_key'))

import chart_studio.plotly as py
import chart_studio.tools as tls

In [11]:
usr = os.environ.get('sql_user')
pwd = os.environ.get('sql_pwd')
host = os.environ.get('sql_host')
db = os.environ.get('sql_db')

def query(sql):
    
    cnx = mysql.connector.connect(user=usr, 
                              password=pwd,
                              host=host,
                              database=db,
                                 port=3306)
    
    df = pd.read_sql(sql, cnx)
    cnx.close()
    return df

In [12]:
sum_mod_day_query = '''
SELECT user_id, date_diff, COUNT(date_diff) as count
FROM
(
SELECT user_id, date_diff
FROM
(
SELECT ecm.user_id, DATEDIFF(ecm.date_created, u.date_created) AS date_diff
FROM education_completed_module ecm
LEFT JOIN user u
ON u.id = ecm.user_id
ORDER BY ecm.user_id
) AS date_table
WHERE date_diff IS NOT NULL
) AS num_mod
GROUP BY user_id, date_diff
'''

sum_mod_day = query(sum_mod_day_query)

sum_mod_day

ProgrammingError: 1045 (28000): Access denied for user ''@'125.129.130.251' (using password: NO)

In [4]:
# Returns a table with the count of the total number 
# of X for each day after registration for all users

def dateSum(tab):
    tab = tab.groupby(['date_diff']).sum()
    
    # Change column to suit table
    tab = tab.drop(columns = ['user_id'])
    tab = tab.reset_index()
    tab['logcount'] = np.log(tab['count'])
    
    return tab

In [59]:
datemod = dateSum(sum_mod_day)

In [60]:
datemod

,date_diff,count,logcount
0,0,212083,12.264733
1,1,81791,11.311922
2,2,39923,10.594708
3,3,26659,10.190882
4,4,19934,9.900182
...,...,...,...
189,189,9,2.197225
190,190,7,1.945910
191,195,2,0.693147
192,196,7,1.945910


In [63]:
fig_sum_mod = px.line(datemod, x = 'date_diff', y = 'logcount', 
                      title = 'Log of Sum of Number of Modules <br>Completed by Days After Registration')

fig_sum_mod.show()

In [65]:
sum_ref_day_query = '''
SELECT referrer_id, date_diff, COUNT(date_diff) as count
FROM
(
SELECT referrer_id, date_diff
FROM
(
SELECT rr.referrer_id, DATEDIFF(rr.date_executed, u.date_created) AS date_diff
FROM referral_relationship rr
LEFT JOIN user u
ON u.id = rr.referrer_id
ORDER BY rr.referrer_id
) AS refer_tab
WHERE date_diff IS NOT NULL
) AS date_tab
GROUP BY referrer_id, date_diff
'''

sum_ref_day = query(sum_ref_day_query)

In [66]:
refdate = dateSum(sum_ref_day)

In [70]:
fig_sum_ref = px.line(refdate, x = 'date_diff', y = 'logcount', 
                      title = 'Log of Sum of Number of Referrals <br>Given Out by Days After Registration')

fig_sum_ref.show()

In [71]:
py.plot(fig_sum_ref, name = "Referrals by Day After Regist", auto_open = True)

'https://plotly.com/~woonggyu.jin/110/'

In [69]:
py.plot(fig_sum_mod, name = "Modules by Day After Regist", auto_open = True)

'https://plotly.com/~woonggyu.jin/108/'

In [83]:
sum_do_day_query = '''
SELECT user_id, date_diff, COUNT(date_diff) as count
FROM
(
SELECT user_id, date_diff
FROM
(
SELECT oc.user_id, DATEDIFF(oc.date_created, u.date_created) AS date_diff
FROM objective_completed oc
LEFT JOIN user u
ON u.id = oc.user_id
WHERE u.date_created >= '2020-04-16'
ORDER BY oc.user_id
) AS do_tab
WHERE date_diff IS NOT NULL
) AS date_tab
GROUP BY user_id, date_diff
'''


sum_do_day = query(sum_do_day_query)

In [84]:
datedo = dateSum(sum_do_day)

In [85]:
datedo

,date_diff,count,logcount
0,0,7133,8.872487
1,1,2301,7.741099
2,2,1559,7.351800
3,3,1226,7.111512
4,4,1030,6.937314
5,5,845,6.739337
6,6,733,6.597146
7,7,606,6.406880
8,8,533,6.278521
9,9,450,6.109248


In [87]:
fig_sum_do = px.line(datedo[:-1], x = 'date_diff', y = 'logcount', 
title = 'Log of Sum of Number of Daily Objectives <br>Completed by Days After Registration')

fig_sum_do.show()

In [89]:
py.plot(fig_sum_do, name = "Daily Objectives by Day After Regist", auto_open = True)

'https://plotly.com/~woonggyu.jin/114/'